In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# تثبيت المكتبات المطلوبة
!pip install easyocr
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 5.7 MB/s eta 0:00:00


In [3]:
# تثبيت المكتبات المطلوبة
!pip install ultralytics
!pip install opencv-python-headless
!pip install easyocr
!pip install paddleocr
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 874.8/874.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.4 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117030 sha256=8fed7de141844fd3d7f1c561e593ddc585ea76f55b8fcdf3c15bb040d751b757
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire


In [ ]:
## this code using easyocr AND Inputs-Video-of-car-plates.mp4
import cv2
import numpy as np
from ultralytics import YOLO
import easyocr
import os
import shutil
from google.colab import drive

## this code show the result and plate bigger

In [11]:
# تحميل النموذجين
car_model_path = "/content/yolov8n.pt"  # مسار نموذج كشف السيارات
license_model_path = "/content/license_plate_detector (2).pt"  # مسار نموذج كشف اللوحات

# تحميل النماذج
car_detector = YOLO(car_model_path)
license_detector = YOLO(license_model_path)

# تحميل قارئ EasyOCR
reader = easyocr.Reader(['en'], gpu=False)

# مسار الفيديو الإدخال
input_video_path = '/content/Inputs-Video-of-car-plates.mp4'  # استبدل هذا المسار بمسار الفيديو الخاص بك

# فتح الفيديو
cap = cv2.VideoCapture(input_video_path)

# الحصول على خصائص الفيديو
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# فتح فيديو الخروج
output_video_path = 'output_video_with_plates.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # تحديد كود الترميز لملف AVI
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# الاتصال بـ Google Drive
drive.mount('/content/drive')
drive_output_path = '/content/drive/MyDrive/output_video_with_plates.avi'  # المسار الذي سيتم حفظ الملف فيه في Google Drive

# قائمة المركبات التي سيتم اكتشافها (السيارات، الشاحنات، ...):
# [2]: السيارات، [3]: الشاحنات الصغيرة، [5]: الشاحنات، [7]: الحافلات
vehicle_classes = [2, 3, 5, 7]

# معالجة كل إطار في الفيديو لمدة ثانية واحدة فقط
frame_count = 0
max_frame_count = int(fps * 1)  # حساب عدد الإطارات في ثانية واحدة

while cap.isOpened() and frame_count < max_frame_count:
    ret, frame = cap.read()
    if not ret:
        break

    # طباعة معلومات الإطار الحالي
    print(f"Processing frame {frame_count + 1}/{max_frame_count}")

    # كشف السيارات باستخدام نموذج YOLO للسيارات
    car_results = car_detector(frame, conf=0.3)[0]  # تقليل درجة الثقة إلى 0.3

    # إذا لم يتم اكتشاف أي سيارة، طباعة رسالة تحذير
    if len(car_results.boxes) == 0:
        print("No cars detected in this frame!")

    # قائمة لحفظ إحداثيات مركز كل مركبة
    license_texts = []

    for car in car_results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = car

        # التحقق من أن الجسم المكتشف هو سيارة أو مركبة أخرى
        if int(class_id) in vehicle_classes:
            # رسم المستطيل حول السيارة
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

            # قص جزء السيارة من الإطار
            car_crop = frame[int(y1):int(y2), int(x1):int(x2)]

            # كشف لوحة الأرقام داخل المنطقة المقصوصة
            license_results = license_detector(car_crop, conf=0.3)[0]  # تقليل درجة الثقة إلى 0.3

            for license_plate in license_results.boxes.data.tolist():
                lx1, ly1, lx2, ly2, lscore, lclass_id = license_plate
                # تحديد الإحداثيات الفعلية في إطار الفيديو الكامل
                abs_x1 = int(x1 + lx1)
                abs_y1 = int(y1 + ly1)
                abs_x2 = int(x1 + lx2)
                abs_y2 = int(y1 + ly2)

                # قص لوحة الأرقام
                plate_crop = frame[abs_y1:abs_y2, abs_x1:abs_x2]

                # تحويل الصورة إلى رمادية
                gray_plate = cv2.cvtColor(plate_crop, cv2.COLOR_BGR2GRAY)

                # قراءة النص باستخدام EasyOCR
                ocr_results = reader.readtext(gray_plate)

                # استخراج النص من النتائج
                plate_text = ""
                for (bbox, text, prob) in ocr_results:
                    plate_text += text + " "

                # رسم المستطيل حول لوحة الأرقام
                cv2.rectangle(frame, (abs_x1, abs_y1), (abs_x2, abs_y2), (0, 255, 0), 2)
                # وضع النص فوق لوحة الأرقام
                cv2.putText(frame, plate_text, (abs_x1, abs_y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                # تكبير لوحة الأرقام وعرضها في الإطار
                if plate_crop.shape[0] > 0 and plate_crop.shape[1] > 0:
                    plate_crop_resized = cv2.resize(plate_crop, (int((abs_x2 - abs_x1) * 3), int((abs_y2 - abs_y1) * 3)))
                    plate_h, plate_w, _ = plate_crop_resized.shape
                    if abs_y1 - plate_h - 10 > 0:
                        frame[abs_y1 - plate_h - 10:abs_y1 - 10, abs_x1:abs_x1 + plate_w] = plate_crop_resized

                license_texts.append(plate_text)

    # كتابة الإطار المعدل إلى فيديو الخروج
    out.write(frame)

    # زيادة عدد الإطارات المعالجة
    frame_count += 1

# إطلاق الموارد
cap.release()
out.release()
cv2.destroyAllWindows()

# نسخ الفيديو الناتج إلى Google Drive
shutil.copy(output_video_path, drive_output_path)

print(f"تم معالجة أول ثانية من الفيديو وحفظها بنجاح في: {drive_output_path}")

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteDrive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing frame 1/30

0: 384x640 19 cars, 1 bus, 2 trucks, 332.9ms
Speed: 15.2ms preprocess, 332.9ms inference, 31.3ms postprocess per image at shape (1, 3, 384, 640)

0: 512x640 1 license_plate, 186.4ms
Speed: 4.2ms preprocess, 186.4ms inference, 0.9ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 1 license_plate, 169.7ms
Speed: 4.8ms preprocess, 169.7ms inference, 0.9ms postprocess per image at shape (1, 3, 512, 640)

0: 576x640 1 license_plate, 213.3ms
Speed: 4.6ms preprocess, 213.3ms inference, 1.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 1 license_plate, 206.6ms
Speed: 4.9ms preprocess, 206.6ms inference, 1.3ms postprocess per image at shape (1, 3, 576, 640)

0: 480x640 (no detections), 167.3ms
Speed: 4.1ms preprocess, 167.3ms inference, 0.7m